In [94]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
import matplotlib as plt
import warnings
import datetime as dt
import pickle

warnings.filterwarnings('ignore')

In [95]:
# data & model load
# 가뭄

tmp = pd.read_csv('../model/가뭄모델data.csv',index_col=0)
drought_x = tmp.drop(['일시','est+60'],axis=1).reset_index(drop=True)
drought_y = tmp['est+60'].apply(lambda x : 1 if x<-1 else 0).reset_index(drop=True)
with open('../model/가뭄모델.pickle','rb') as f:
    drought_xgbc = pickle.load(f)

In [96]:
# 장마
tmp = pd.read_csv('../model/장마_test_2019.csv',index_col=0)
rain_x = tmp.drop(['일시','target_t0'],axis=1).reset_index(drop=True)
rain_y = tmp['target_t0'].reset_index(drop=True)
with open('../model/장마.pickle','rb') as f:
    rain_lgbm = pickle.load(f)

In [152]:
# ai
ai_x = pd.read_csv('../model/ai모델xdata.csv',index_col=0).reset_index(drop=True)
ai_y = pd.read_csv('../model/ai모델ydata.csv',index_col=0).reset_index(drop=True)['est_target']
with open('../model/ai모델.pickle','rb') as f:
    ai_svc = pickle.load(f)

In [98]:
# 열대야
tmp = pd.read_csv('../model/열대야2019.csv',index_col=0)
hott_x = tmp.drop(['재난일어난날'],axis=1).reset_index(drop=True)
hott_y = tmp['재난일어난날'].reset_index(drop=True)
with open('../model/열대야rfc.pickle','rb') as f:
    hott_rfc = pickle.load(f)

In [99]:
benefit_li=[]

def stock(money, stock_code, buy_date, sell_date):
    df_stock = fdr.DataReader(stock_code,'2019','2020')
    df_stock.reset_index(inplace=True)

    while len(df_stock[df_stock['Date']==buy_date]) == 0:
        buy_date-= dt.timedelta(days=1)
        if buy_date.year < 2019:
            print('error: 2019년도 내에 구입 불가')
            return 0
    buy_closing_price = df_stock[df_stock['Date']==buy_date]['Close'].values[0]
    buy_num = int(money/buy_closing_price)
    buy_price = buy_closing_price * buy_num

    while len(df_stock[df_stock['Date']==sell_date]) == 0:
        sell_date-= dt.timedelta(days=1)
        if sell_date == buy_date:
            print('error: 2019년도 내에 구입 불가')
            return 0
    sell_closing_price = df_stock[df_stock['Date']==sell_date]['Close'].values[0]
    sell_price = sell_closing_price * buy_num
    margin = sell_price - buy_price

    benefit = (sell_closing_price/buy_closing_price) * 100 - 100
    print(f"매입 : {buy_date.strftime('%Y-%m-%d')}, 매도 : {sell_date.strftime('%Y-%m-%d')}")
    print('주식수 : ', buy_num)
    print('매입단가 : ', buy_closing_price,'원')
    print('매입가 : ', buy_price, '원') 
    print('매도단가 : ', sell_closing_price, '원')
    print('매도가 : ', sell_price, '원') 
    print('수익금액 : ', margin, '원')
    print('수익률 : ', '{:.2f} %'.format(benefit)) 
    return margin

In [153]:
stock_lst = {'장마':[{'폐기물주':'151860,013810,067900,060150,029960',
                         '비료주':'002100,025860,007590,001550',
                         '제습기주':'002700,044340,021240,037070',},
                   (rain_lgbm, rain_x, rain_y),
                  {'폐기물주':[30, 7],'비료주':[21, 7],'제습기주':[28, 14]},
                   30],
             '열대야':[{'식료품주':'071840,005180,006890',
                    '에어컨주':'086670,002700,042110,058610,044340,037070',},
                   (hott_rfc, hott_x, hott_y),
                   {'에어컨주':[14,3],'식료품주':[30,5]},
                   14],
             '가뭄':[{'수도관주':'126870,060260,008970,076080,043910'},
                  (drought_xgbc, drought_x, drought_y),
                  {'수도관주':[60,30]},
                   60],
             'ai':[{'사료주':'006980,086060,005860',
                   '의료,백신주':'065170,036480,054670,044960,052670,072020,043200',
                   '대체식품주':'011150,030720,025870',},
                  (ai_svc, ai_x, ai_y),
                   {'사료주':[2,5],'의료,백신주':[2,5],'대체식품주':[2,5]},
                   7]
            }

In [158]:
money = []
dlst = ['장마','열대야','가뭄','ai']

for disaster in dlst:
    model,x,y = stock_lst[disaster][1]
    for theme in stock_lst[disaster][0].keys():
        money.append([theme, [1000000 for _ in range(len(stock_lst[disaster][0][theme].split(',')))]])
        est = stock_lst[disaster][3]
        today = dt.datetime(2019,1,1)
        sell_date = dt.datetime(2019,1,1)
        flag = 0
        # 1년 루프
        for day in range(len(x)):
            today += dt.timedelta(days=1)
            if today < sell_date: # 매입을 해서 자금이 없는경우 다음 날로 패스
                continue
            if flag > 0: # flag 값을 가지고 있다면 감소시키며 다음 날로 넘어감
                flag -= 1 # 어떤 이벤트도 없는 날에만 모델 판단 하기 위한 코드
                if y.iloc[day] == 1: # 재난이 벌어지고 있는 경우 flag 초기화
                    flag = 30
                continue
            if model.predict(x.iloc[[0,day],:])[1]==1: # 미래에 재난이 발생할거라 예측하면
                flag = 30 # 플래그 부여
                buy_date = today+dt.timedelta(days=est-stock_lst[disaster][2][theme][0])
                sell_date = today+dt.timedelta(days=est+stock_lst[disaster][2][theme][1])
                for i, code in enumerate(stock_lst[disaster][0][theme].split(',')):
                    money[-1][1][i] += stock(money[-1][1][i], code, buy_date, sell_date)
                # 투자 전략에 의거 구매 일자랑 판매일자를 사전에 지정해두고 그 날짜까지 스톱


매입 : 2019-05-28, 매도 : 2019-07-04
주식수 :  279
매입단가 :  3580 원
매입가 :  998820 원
매도단가 :  3900 원
매도가 :  1088100 원
수익금액 :  89280 원
수익률 :  8.94 %
매입 : 2019-05-28, 매도 : 2019-07-04
주식수 :  316
매입단가 :  3155 원
매입가 :  996980 원
매도단가 :  3420 원
매도가 :  1080720 원
수익금액 :  83740 원
수익률 :  8.40 %
매입 : 2019-05-28, 매도 : 2019-07-04
주식수 :  124
매입단가 :  8050 원
매입가 :  998200 원
매도단가 :  8540 원
매도가 :  1058960 원
수익금액 :  60760 원
수익률 :  6.09 %
매입 : 2019-05-28, 매도 : 2019-07-04
주식수 :  115
매입단가 :  8650 원
매입가 :  994750 원
매도단가 :  8010 원
매도가 :  921150 원
수익금액 :  -73600 원
수익률 :  -7.40 %
매입 : 2019-05-28, 매도 : 2019-07-04
주식수 :  88
매입단가 :  11350 원
매입가 :  998800 원
매도단가 :  11350 원
매도가 :  998800 원
수익금액 :  0 원
수익률 :  0.00 %
매입 : 2019-06-05, 매도 : 2019-07-04
주식수 :  61
매입단가 :  16350 원
매입가 :  997350 원
매도단가 :  16900 원
매도가 :  1030900 원
수익금액 :  33550 원
수익률 :  3.36 %
매입 : 2019-06-05, 매도 : 2019-07-04
주식수 :  101
매입단가 :  9890 원
매입가 :  998890 원
매도단가 :  10950 원
매도가 :  1105950 원
수익금액 :  107060 원
수익률 :  10.72 %
매입 : 2019-06-05, 매도 : 2019-07-04
주식수 :  

In [159]:
money

[['폐기물주', [1089280, 1083740, 1060760, 926400, 1000000]],
 ['비료주', [1033550, 1107060, 1015400, 874200]],
 ['제습기주', [828640, 894400, 1009600, 1181192]],
 ['식료품주', [835000, 934200, 1044980]],
 ['에어컨주', [844960, 802510, 833995, 1023180, 914900, 776080]],
 ['수도관주', [745900, 954430, 887005, 956635, 765448]],
 ['사료주', [1037900, 1061980, 1123420]],
 ['의료,백신주', [1124315, 1032250, 1038500, 927575, 1102231, 999800, 976150]],
 ['대체식품주', [984405, 1023650, 1032340]]]